In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from os import listdir
from os.path import isfile, join
from Bio import AlignIO,SeqIO
from Bio.Alphabet import generic_dna
import paramiko
from os.path import isfile, join
import urllib
import re

# for Genes

In [2]:
path="/Users/maeperez/Desktop/HYPHY/GROUP1/"

onlyfiles_mac = [path+f for f in listdir(path) if f[-7:]=='.KHtest']

print len(onlyfiles_mac), onlyfiles_mac[0]
# print [fi.split('_') for fi in onlyfiles_all]
# onlyfiles= [fi for fi in onlyfiles_all if fi.split('_')[2][:4] in ['0001','0002','0003','0392']]
# print onlyfiles

ssh_client = paramiko.SSHClient()
ssh_client.load_system_host_keys()
ssh_client.connect('srv01775.soton.ac.uk',username='mp2c18',password='Mon2real')
ssh_client.open_sftp()
sftp_client = ssh_client.open_sftp()


path_srv="/home/mp2c18/data/hyphy/absrel/"
onlyfiles_srv=[path_srv+f for f in sftp_client.listdir(path_srv) if f[-7:]=='.HKtest']

647 /Users/maeperez/Desktop/HYPHY/GROUP1/aligned_Rmag_0386.GTR.KHtest


/Users/maeperez/anaconda2/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a release (2.7.7+) that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,
/Users/maeperez/anaconda2/lib/python2.7/site-packages/paramiko/ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo
/Users/maeperez/anaconda2/lib/python2.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point enc

In [3]:
print len(onlyfiles_srv)+len(onlyfiles_mac)
# print onlyfiles_srv[0],onlyfiles_mac[0]
onlyfiles_all=onlyfiles_srv+onlyfiles_mac
srv=[str(f.split('/')[-1].split('.')[0]) for f in onlyfiles_srv]
mac=[str(f.split('/')[-1].split('.')[0]) for f in onlyfiles_mac]
# print [x for x in srv if x in set(mac)]

KHtests=[str(f.split('/')[-1:]).split('.')[0] for f in onlyfiles_all]
KH_list=[f.split('.')[0] for f in sftp_client.listdir(path_srv) if f[-7:]=='.HKtest']
# print KH_list[0]

path_aln="/Users/maeperez/Desktop/Mauve/Clams/GROUP1/group1_fna/"
aln_files=[path_aln+f for f in listdir(path_aln) if f[-4:]=='.fna']
# print [f.split('.')[0] for f in listdir(path_aln) if f[-4:]=='.fna'][0]
aln_list=[f.split('.')[0] for f in listdir(path_aln) if f[-4:]=='.fna']
# print [f for f in KH_list if f not in aln_list]
# print [f for f in aln_list if f not in KH_list]
print onlyfiles_all[:2]
rerunfiles=['/Users/maeperez/Desktop/HYPHY/GROUP1/aligned_Rmag_0178.GTR.KHtest']

717
[u'/home/mp2c18/data/hyphy/absrel/aligned_Rmag_0178.GTR.HKtest', u'/home/mp2c18/data/hyphy/absrel/aligned_Rmag_0012.GTR.HKtest']


In [5]:
kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])
onlyfiles=onlyfiles_all[:]
for i in range(len(onlyfiles[:])):
# for i in range(len(rerunfiles)):
    
    
    ### READ the KHtest file and see if there are significant breakpoints
    if 'mp2c18' in onlyfiles[i]:
        f=sftp_client.open(onlyfiles[i])
    else:
        f=open(onlyfiles[i])
    gene=re.split('/|\.|aligned_',onlyfiles[i])[-3]
    lines=f.read().splitlines()
    f.close()
    
    breakpoints=[0]
    p001=0
    p005=0
    p01=0

    for n in range(len(lines)-20,len(lines)):
        if lines[n].split(' | ')[0]=='Breakpoint':
            for i in range(n+1,len(lines)):
                try:
                    pval_LHS=float(lines[i].split(' | ')[2])
                    pval_RHS=float(lines[i].split(' | ')[4])
                    
                    if pval_LHS<0.01 and pval_RHS<0.01:
                        p001+=1
                    if pval_LHS<0.05 and pval_RHS<0.05:
                        p005+=1
                        breakpoints+=[int(lines[i].split(' | ')[0])]
                    if pval_LHS<0.1 and pval_RHS<0.1:
                        p01+=1

                except:
                    pass

    newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
    kh=kh.append(newentry,ignore_index=True)
    
#     ## READ the corresponding FNA FILE AND BREAK IT UP AT BREAKPOINTS ##

#     alignment = AlignIO.read(path_aln+'aligned_'+gene+'.fna', "fasta",alphabet=generic_dna)

#     breakpoints+=[alignment.get_alignment_length()]
#     start=breakpoints.pop(0)

#     sub=0
#     while len(breakpoints)>0:
#         sub+=1
#         end=breakpoints.pop(0)
#         print start,end
#         new_aln= alignment[:,start:end]
# #         print new_aln
#         for record in new_aln:
#             record.id = record.id.split('|')[0]
        
#         new_aln.sort()
#         filename='aligned_'+gene+'_'+str(sub)+'_'+str(start)+'to'+str(end)+'.nex'
#         print filename
#         AlignIO.write([new_aln], 'genes/nex/'+filename, "nexus")
        
#         start=end
#     else:
kh.to_csv('genes/KHtest_splits_summary.txt',sep='\t',header=True,index=False)     

# LCBs Something wierd: files .parsed_fna not same length as .fna files. My fasta parsing code was wrong. I made it right below

## Parse mafft alignment files prior gard run

In [131]:
# GOOD was of parsing the mafft alignment. the other just doubled each line of the file increasing the length of the sequences

path="/Users/maeperez/Desktop/Mauve/Clams/ALL/core_ALL_mafft3/"

onlyfiles = [path+f for f in listdir(path)]

name='700fasta_mafft'
for file in onlyfiles[:]:
    name=file.split('/')[-1]
    name="{:03d}".format(int(name.replace('fasta_mafft','')))
    alignment = AlignIO.read(file, "fasta",alphabet=generic_dna)
    recs = [record for record in alignment]
    new_recs=[]
    # Add ids
    for rec in recs:
        rec.id=rec.id.split('/')[-2][:-4].replace('.','_')
        rec.description=''

    with open("/Users/maeperez/Desktop/gard/fna/"+name+".parsed_fna", "w") as output_handle:
        SeqIO.write(recs, output_handle, "fasta")


## After running gard (on the NOC server), parse results; splits LCBs on breakpoints

In [31]:
path_KHtest="/home/mp2c18/data/hyphy/gard/gard_out/"
onlyfiles_srv=sorted([path_KHtest+f for f in sftp_client.listdir(path_KHtest) if f[-7:]=='.KHtest'])
path_aln="/home/mp2c18/data/hyphy/gard/aligned_core_LCB/"
path_out='/Users/maeperez/Desktop/MyBayes_Bucky/LCB/'


onlyfiles_srv[:5]

[u'/home/mp2c18/data/hyphy/gard/gard_out/LCB001.GTR.KHtest',
 u'/home/mp2c18/data/hyphy/gard/gard_out/LCB002.GTR.KHtest',
 u'/home/mp2c18/data/hyphy/gard/gard_out/LCB003.GTR.KHtest',
 u'/home/mp2c18/data/hyphy/gard/gard_out/LCB004.GTR.KHtest',
 u'/home/mp2c18/data/hyphy/gard/gard_out/LCB005.GTR.KHtest']

In [36]:
kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])

for s in range(len(onlyfiles_srv[:])):
    
    ### READ the KHtest file and see if there are significant breakpoints
    
    f=sftp_client.open(onlyfiles_srv[s])
    gene=re.split('/|\.|aligned_',onlyfiles_srv[s])[-3]
    lines=f.read().splitlines()
    f.close()
    
    breakpoints=[0]
    p001=0
    p005=0
    p01=0

    for n in range(len(lines)-20,len(lines)):
        if lines[n].split(' | ')[0]=='Breakpoint':
            for i in range(n+1,len(lines)):
                try:
                    pval_LHS=float(lines[i].split(' | ')[2])
                    pval_RHS=float(lines[i].split(' | ')[4])
                    
                    if pval_LHS<0.01 and pval_RHS<0.01:
                        p001+=1
                    if pval_LHS<0.05 and pval_RHS<0.05:
                        p005+=1
                        breakpoints+=[int(lines[i].split(' | ')[0])]
                    if pval_LHS<0.1 and pval_RHS<0.1:
                        p01+=1

                except:
                    pass


    newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
    kh=kh.append(newentry,ignore_index=True)

    ## READ the corresponding FNA FILE AND BREAK IT UP AT BREAKPOINTS ##
    print path_aln+gene+'.parsed_fna'
    with sftp_client.open(path_aln+gene+'.parsed_fna', "r") as f:
        alignment = AlignIO.read(f, "fasta",alphabet=generic_dna)

    breakpoints+=[alignment.get_alignment_length()]
    start=breakpoints.pop(0)
    print breakpoints
    sub=0
    while len(breakpoints)>0:
        sub+=1
        end=breakpoints.pop(0)
#         print start,end
        new_aln= alignment[:,start:end]

        for record in new_aln:
            record.id = record.id.split('|')[0]
        
        new_aln.sort()
#         print new_aln
        filename=gene+'_'+str(sub)+'_'+str(start)+'to'+str(end)+'.nex'
        print s+1,filename
        
        
        
        AlignIO.write([new_aln],path_out+'nex/'+filename, "nexus")
        
        start=end
#     else:

kh.to_csv(path_out+'KHtest_splits_summary.txt',sep='\t',header=True,index=False)
    


/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB001.parsed_fna
[2420]
1 LCB001_1_0to2420.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB002.parsed_fna
[164]
2 LCB002_1_0to164.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB003.parsed_fna
[722]
3 LCB003_1_0to722.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB004.parsed_fna
[207, 993]
4 LCB004_1_0to207.nex
4 LCB004_2_207to993.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB005.parsed_fna
[2133]
5 LCB005_1_0to2133.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB006.parsed_fna
[649]
6 LCB006_1_0to649.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB007.parsed_fna
[1354]
7 LCB007_1_0to1354.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB008.parsed_fna
[1001]
8 LCB008_1_0to1001.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB009.parsed_fna
[118]
9 LCB009_1_0to118.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB010.parsed_fna
[348, 709]
10 LCB010_1_0to348.nex
10 LCB010_2_348to709.nex
/home/mp2c18

[453]
86 LCB086_1_0to453.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB087.parsed_fna
[554]
87 LCB087_1_0to554.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB088.parsed_fna
[472]
88 LCB088_1_0to472.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB089.parsed_fna
[1573]
89 LCB089_1_0to1573.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB090.parsed_fna
[245, 695]
90 LCB090_1_0to245.nex
90 LCB090_2_245to695.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB091.parsed_fna
[641]
91 LCB091_1_0to641.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB092.parsed_fna
[315]
92 LCB092_1_0to315.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB093.parsed_fna
[235]
93 LCB093_1_0to235.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB094.parsed_fna
[738]
94 LCB094_1_0to738.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB095.parsed_fna
[1051]
95 LCB095_1_0to1051.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB096.parsed_fna
[2371]
96 LCB096_1_0to2371.nex
/home

/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB171.parsed_fna
[584]
171 LCB171_1_0to584.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB172.parsed_fna
[1834]
172 LCB172_1_0to1834.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB173.parsed_fna
[3011]
173 LCB173_1_0to3011.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB174.parsed_fna
[266]
174 LCB174_1_0to266.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB175.parsed_fna
[124]
175 LCB175_1_0to124.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB176.parsed_fna
[605, 1151]
176 LCB176_1_0to605.nex
176 LCB176_2_605to1151.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB177.parsed_fna
[1230]
177 LCB177_1_0to1230.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB178.parsed_fna
[347]
178 LCB178_1_0to347.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB179.parsed_fna
[2139]
179 LCB179_1_0to2139.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB180.parsed_fna
[1241]
180 LCB180_1_0to1241.nex
/home/mp2c18/data

/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB254.parsed_fna
[225]
254 LCB254_1_0to225.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB255.parsed_fna
[1165]
255 LCB255_1_0to1165.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB256.parsed_fna
[301]
256 LCB256_1_0to301.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB257.parsed_fna
[394]
257 LCB257_1_0to394.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB258.parsed_fna
[423]
258 LCB258_1_0to423.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB259.parsed_fna
[342]
259 LCB259_1_0to342.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB260.parsed_fna
[145]
260 LCB260_1_0to145.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB261.parsed_fna
[554]
261 LCB261_1_0to554.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB262.parsed_fna
[249]
262 LCB262_1_0to249.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB263.parsed_fna
[739]
263 LCB263_1_0to739.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB264.parse

[1033]
337 LCB337_1_0to1033.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB338.parsed_fna
[267]
338 LCB338_1_0to267.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB339.parsed_fna
[1261]
339 LCB339_1_0to1261.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB340.parsed_fna
[423, 729]
340 LCB340_1_0to423.nex
340 LCB340_2_423to729.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB341.parsed_fna
[1781]
341 LCB341_1_0to1781.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB342.parsed_fna
[905]
342 LCB342_1_0to905.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB343.parsed_fna
[648]
343 LCB343_1_0to648.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB344.parsed_fna
[892]
344 LCB344_1_0to892.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB345.parsed_fna
[123]
345 LCB345_1_0to123.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB346.parsed_fna
[247]
346 LCB346_1_0to247.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB347.parsed_fna
[3875]
347 LCB347_1_0to

[2994]
422 LCB422_1_0to2994.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB423.parsed_fna
[816]
423 LCB423_1_0to816.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB424.parsed_fna
[2352]
424 LCB424_1_0to2352.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB425.parsed_fna
[198]
425 LCB425_1_0to198.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB426.parsed_fna
[366]
426 LCB426_1_0to366.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB427.parsed_fna
[3105]
427 LCB427_1_0to3105.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB428.parsed_fna
[651]
428 LCB428_1_0to651.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB429.parsed_fna
[2741]
429 LCB429_1_0to2741.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB430.parsed_fna
[431]
430 LCB430_1_0to431.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB431.parsed_fna
[1261]
431 LCB431_1_0to1261.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB432.parsed_fna
[509]
432 LCB432_1_0to509.nex
/home/mp2c18/data/hy

[383]
506 LCB506_1_0to383.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB507.parsed_fna
[621]
507 LCB507_1_0to621.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB508.parsed_fna
[150, 286]
508 LCB508_1_0to150.nex
508 LCB508_2_150to286.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB509.parsed_fna
[170]
509 LCB509_1_0to170.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB510.parsed_fna
[735]
510 LCB510_1_0to735.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB511.parsed_fna
[684]
511 LCB511_1_0to684.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB512.parsed_fna
[382]
512 LCB512_1_0to382.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB513.parsed_fna
[124, 257, 547]
513 LCB513_1_0to124.nex
513 LCB513_2_124to257.nex
513 LCB513_3_257to547.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB514.parsed_fna
[1076]
514 LCB514_1_0to1076.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB515.parsed_fna
[947, 2622]
515 LCB515_1_0to947.nex
515 LCB515_2_947to2622.

[736]
588 LCB588_1_0to736.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB589.parsed_fna
[284]
589 LCB589_1_0to284.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB590.parsed_fna
[2094]
590 LCB590_1_0to2094.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB591.parsed_fna
[524]
591 LCB591_1_0to524.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB592.parsed_fna
[8103]
592 LCB592_1_0to8103.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB593.parsed_fna
[2768]
593 LCB593_1_0to2768.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB594.parsed_fna
[551]
594 LCB594_1_0to551.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB595.parsed_fna
[228]
595 LCB595_1_0to228.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB596.parsed_fna
[1564]
596 LCB596_1_0to1564.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB597.parsed_fna
[285]
597 LCB597_1_0to285.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB598.parsed_fna
[1440]
598 LCB598_1_0to1440.nex
/home/mp2c18/data/hy

[2699]
672 LCB672_1_0to2699.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB673.parsed_fna
[1898]
673 LCB673_1_0to1898.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB674.parsed_fna
[1023]
674 LCB674_1_0to1023.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB675.parsed_fna
[1690]
675 LCB675_1_0to1690.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB676.parsed_fna
[425]
676 LCB676_1_0to425.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB677.parsed_fna
[1960]
677 LCB677_1_0to1960.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB678.parsed_fna
[850]
678 LCB678_1_0to850.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB679.parsed_fna
[1066]
679 LCB679_1_0to1066.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB680.parsed_fna
[793]
680 LCB680_1_0to793.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB681.parsed_fna
[216]
681 LCB681_1_0to216.nex
/home/mp2c18/data/hyphy/gard/aligned_core_LCB/LCB682.parsed_fna
[194]
682 LCB682_1_0to194.nex
/home/mp2c18/data/